In [1]:
spark

In [3]:
! wget https://github.com/kitwaicloud/kitwaicloud.github.io/raw/master/dataset/churn-bigml-80.csv

--2018-12-17 20:05:33--  https://github.com/kitwaicloud/kitwaicloud.github.io/raw/master/dataset/churn-bigml-80.csv
Resolving github.com (github.com)... 192.30.253.112, 192.30.253.113
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/kitwaicloud/kitwaicloud.github.io/master/dataset/churn-bigml-80.csv [following]
--2018-12-17 20:05:33--  https://raw.githubusercontent.com/kitwaicloud/kitwaicloud.github.io/master/dataset/churn-bigml-80.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 223998 (219K) [text/plain]
Saving to: ‘churn-bigml-80.csv’

churn-bigml-80.csv  100%[===================>] 218.75K  --.-KB/s    in 0.04s   

2018-12-17 20:05:34 (5.27 M

In [22]:
! wget https://github.com/kitwaicloud/kitwaicloud.github.io/raw/master/dataset/churn-bigml-20.csv

--2018-12-17 20:14:00--  https://github.com/kitwaicloud/kitwaicloud.github.io/raw/master/dataset/churn-bigml-20.csv
Resolving github.com (github.com)... 192.30.253.112, 192.30.253.113
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/kitwaicloud/kitwaicloud.github.io/master/dataset/churn-bigml-20.csv [following]
--2018-12-17 20:14:00--  https://raw.githubusercontent.com/kitwaicloud/kitwaicloud.github.io/master/dataset/churn-bigml-20.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56329 (55K) [text/plain]
Saving to: ‘churn-bigml-20.csv’

churn-bigml-20.csv  100%[===================>]  55.01K  --.-KB/s    in 0.02s   

2018-12-17 20:14:00 (2.63 MB/

In [4]:
ls

bin/                              Miniconda3-4.5.4-Linux-x86_64.sh*
boot/                             mnt/
churn-bigml-80.csv                opt/
dataproc-initialization-actions/  proc/
dev/                              root/
etc/                              run/
hadoop/                           sbin/
home/                             srv/
initrd.img@                       sys/
initrd.img.old@                   tmp/
lib/                              usr/
lib64/                            var/
lost+found/                       vmlinuz@
media/                            vmlinuz.old@
miniconda@


In [23]:
! hdfs dfs -mkdir dataset
! hdfs dfs -put churn-bigml-80.csv dataset
! hdfs dfs -put churn-bigml-20.csv dataset

mkdir: `dataset': File exists
put: `dataset/churn-bigml-80.csv': File exists


In [33]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

CV_data = sqlContext.read.load('hdfs://cluster-c343-m/user/root/dataset/churn-bigml-80.csv', 
                          format='com.databricks.spark.csv', 
                          header='true', 
                          inferSchema='true').cache()

from pyspark.sql.types import DoubleType, StringType
from pyspark.sql.functions import UserDefinedFunction

toStr = UserDefinedFunction(lambda k: k, StringType())
CV_data = CV_data.withColumn('Churn', toStr(CV_data['Churn']))

In [29]:
predictors=('Number vmail messages',
 'Total day minutes',
 'Total day calls',
 'Total eve minutes',
 'Total eve calls',
 'Total night minutes',
 'Total night calls',
 'Total intl minutes',
 'Total intl calls',
 'Customer service calls',
 'IntlPlan',
 'VmailPlan')

In [32]:
lispred=['Number vmail messages',
 'Total day minutes',
 'Total day calls']
CV_data.select(lispred).show(3)


+---------------------+-----------------+---------------+
|Number vmail messages|Total day minutes|Total day calls|
+---------------------+-----------------+---------------+
|                   25|            265.1|            110|
|                   26|            161.6|            123|
|                    0|            243.4|            114|
+---------------------+-----------------+---------------+
only showing top 3 rows



In [34]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier

from pyspark.ml.classification import RandomForestClassifier

# Index labels, adding metadata to the label column
stringindexer = StringIndexer(inputCol='Churn',
                             outputCol='indexedLabel')

stringindexerIntlPlan = StringIndexer(inputCol='International plan',
                             outputCol='IntlPlan')

stringindexerVmailPlan = StringIndexer(inputCol='Voice mail plan',
                             outputCol='VmailPlan')

assembler=VectorAssembler(inputCols=predictors,outputCol='features')

# Train a RandomForest model.
rf_algorithm = RandomForestClassifier(\
                                      labelCol='indexedLabel', featuresCol='features')

# Train a DecisionTree model
#dTree_algorithm = DecisionTreeClassifier(maxDepth=2,
#                                        labelCol='indexedLabel', featuresCol='features')


In [35]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[stringindexer,\
                            stringindexerIntlPlan,\
                            stringindexerVmailPlan,\
                            assembler, rf_algorithm])

#                            assembler, dTree_algorithm])

In [36]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

evaluator=BinaryClassificationEvaluator(labelCol='indexedLabel',\
                                        rawPredictionCol='rawPrediction',\
                                       metricName='areaUnderROC')


# Search through decision tree's maxDepth parameter for best model
#paramGrid = ParamGridBuilder().addGrid(dTree_algorithm.maxDepth, [2,3,4,5,6,7]).build()
paramGrid = ParamGridBuilder().addGrid(rf_algorithm.numTrees, [100,200,400,800])\
                        .addGrid(rf_algorithm.maxDepth, [2,3,4,5,6,7])\
                        .build()

# Set up 3-fold cross validation
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=3)

In [37]:
from time import time

t0 = time()

CrossvalModel=crossval.fit(CV_data)

tt = time() - t0
print("Task completed in {} seconds".format(round(tt,3)))


Task completed in 409.615 seconds


### <font color=red>Evaluation on TEST data

In [39]:

Test_data = sqlContext.read.load('hdfs://cluster-c343-m/user/root/dataset/churn-bigml-20.csv', 
                          format='com.databricks.spark.csv', 
                          header='true', 
                          inferSchema='true').cache()


toStr = UserDefinedFunction(lambda k: k, StringType())
Test_data = Test_data.withColumn('Churn', toStr(Test_data['Churn']))

In [40]:
# make predictions and evaluate result
predictions_Test = CrossvalModel.transform(Test_data)
accuracy_Test=evaluator.evaluate(predictions_Test)

print(accuracy_Test)

0.9301987486197972


In [41]:
# Confussion Matrix
predictions_Test.crosstab('Churn','prediction').show()

+----------------+---+---+
|Churn_prediction|0.0|1.0|
+----------------+---+---+
|           false|570|  2|
|            true| 33| 62|
+----------------+---+---+



In [11]:
# make predictions and evaluate result
#pipelineModel=pipeline.fit(CV_data)
#predictions_Test = pipelineModel.transform(Test_data)
#accuracy_Test=evaluator.evaluate(predictions_Test)

#print(accuracy_Test)

### <font color=red>Evaluation on <font color=green> TRAIN data

In [42]:
# make predictions and evaluate result
predictions_Train = CrossvalModel.transform(CV_data)

accuracy_Train=evaluator.evaluate(predictions_Train)

print(accuracy_Train)

0.9734755573255458


In [43]:
# Confussion Matrix
predictions_Train.crosstab('Churn','prediction').show()

+----------------+----+---+
|Churn_prediction| 0.0|1.0|
+----------------+----+---+
|           false|2278|  0|
|            true|  87|301|
+----------------+----+---+



In [15]:
#pipelineModel=pipeline.fit(CV_data)
#predictions_Train = pipelineModel.transform(CV_data)
#accuracy_Train=evaluator.evaluate(predictions_Train)

#print(accuracy_Train)